In [89]:
import os, sys

sys.path.insert(0, os.pardir)


import cv2

import numpy as np


In [90]:
def load_image():
    image_path = os.path.join(os.pardir, 'assets', 'inklabels.png')
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return image
image = load_image()
image.shape

(8181, 6330)

In [91]:
def get_padding(mask_shape, tile_size=256, overlap=1):
    pad_left = int(overlap * tile_size)
    pad_up = int(overlap * tile_size)
    pad_right = int(overlap * tile_size + tile_size - mask_shape[1] % tile_size)
    pad_down = int(overlap * tile_size + tile_size - mask_shape[0] % tile_size)
    padding = [(pad_up, pad_down), (pad_left, pad_right)]

    return padding

padding = get_padding(image.shape)
image_pad = np.pad(image, padding)
image_pad.shape

(8704, 6912)

In [92]:
def create_items(mask_pad, tile_size=256, overlap=1):
    items = []
    overlap_size = int(overlap * tile_size)
    x_list = np.arange(0, mask_pad.shape[1] - overlap_size, overlap_size).tolist()
    y_list = np.arange(0, mask_pad.shape[0] - overlap_size, overlap_size).tolist()

    for x in x_list:
        for y in y_list:
            bbox =[x, y, x + tile_size, y + tile_size]
            x0, y0, x1, y1 = bbox
            tile = mask_pad[y0:y1, x0:x1]

            if tile.sum() / (255 * tile_size ** 2) >=  0.05:
                items.append(bbox)

    return items

list_bbox = create_items(image_pad)
len(list_bbox)

228

In [96]:
# image_rgb = cv2.cvtColor(image_pad, cv2.COLOR_GRAY2RGB)
# display(image_rgb.shape)
image_rgb = cv2.cvtColor(image_pad, cv2.COLOR_GRAY2RGB)
for i, bbox in enumerate(list_bbox):
    x_min, y_min, x_max, y_max = bbox
    thickness = 35
    cv2.rectangle(image_rgb, (x_min+thickness, y_min+thickness), (x_max+thickness, y_max+thickness), (186, 86, 15), thickness=thickness)
    image_resized = cv2.resize(image_rgb, (int(image_rgb.shape[0] / 16), int(image_rgb.shape[1] / 10)))
    cv2.imwrite(f'../assets/tilled/images_{i}.png', image_resized)

In [94]:
image_resized = cv2.resize(image_rgb, (int(image_rgb.shape[0] / 16), int(image_rgb.shape[1] / 10)))
display(image_resized.shape)
cv2.imwrite('../assets/tilled_images.png', image_resized)

(691, 544, 3)

True